<a href="https://colab.research.google.com/github/SrBliss/CometML-NLP_IMDB/blob/main/Comet_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing required packages

In [1]:
!pip install comet_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412 kB 14.6 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 495 kB 54.3 MB/s 
     |████████████████████████████████| 54 kB 1.5 MB/s 
     |████████████████████████████████| 162 kB 49.3 MB/s 
     |████████████████████████████████| 140 kB 21.6 MB/s 
     |████████████████████████████████| 127 kB 47.2 MB/s 
     |████████████████████████████████| 162 kB 13.5 MB/s 
     |████████████████████████████████| 158 kB 47.8 MB/s 
     |████████████████████████████████| 157 kB 5.1 MB/s 
     |████████████████████████████████| 157 kB 43.4 MB/s 
     |████████████████████████████████| 157 kB 41.5 MB/s 
     |████████████████████████████████| 157 kB 44.0 MB/s 
     |████████████████████████████████| 157 kB 41.5 MB/s 
     |████████████████████████████████| 157 kB 11.0 MB/s 
     |██████████████████████████████

## Importing required packages

In [2]:
# import logging
import os

from comet_ml import Experiment
from comet_ml import config
from comet_ml import API

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Load data and create partitions (train, validation, test)

*This cell takes about 1 minute to execute*

In [3]:
# Load data
train_data, test_data = tfds.load(name="imdb_reviews",
                                  split=["train", "test"], 
                                  batch_size=-1, 
                                  as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)
    
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]
y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQUBFHE/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQUBFHE/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQUBFHE/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## My Experiment: Build, Train and Track the Movie Reviews Model 

In [ ]:
# !cat /proc/cpuinfo

In [ ]:
# TO DO: API on config file or on environment variable
# os.environ['COMET_API_KEY'] = "EpnUTFvwqbgvmSpjWOTMBPlP5"

In [19]:
exp = Experiment (
    api_key="EpnUTFvwqbgvmSpjWOTMBPlP5",
    project_name="rmiranda-movie-reviews",
    workspace="rodrigomiranda21",
    # Some extra optional parameters below
    auto_histogram_weight_logging=True,
    log_code=True,
    )

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/rodrigomiranda21/rmiranda-movie-reviews/3aad1e6f01b246fd8d4ad8f3a2cab846



In [20]:
 params = {
     'layer-1-size': 16,
     'epochs': 10,
     'batch-size':512,
     'dropout': 0.2,
     }

# This will be logged in the Hyperparameters Tab in the Comet UI
exp.log_parameters(params)

**Load Model**

The model will be used for fine tuning. It will be the first "layer" of our NN.

In [21]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(model, 
                           output_shape=[20], 
                           input_shape=[], 
                           dtype=tf.string, 
                           trainable=True)

In [ ]:
# hub_layer(train_examples[:3])

**Build Model**

Create Keras NN - Model + layer-1 + final layer (size 1) 

In [22]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(exp.get_parameter('layer-1-size'), activation='relu'))
model.add(tf.keras.layers.Dropout(exp.get_parameter('dropout')))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_2 (Dense)             (None, 16)                336       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


**Train Model**

In [23]:
# After training the model you will see new Charts, Panels, Hyperparameters, Metrics and more in the Comet UI

model.fit(partial_x_train,
          partial_y_train,
          epochs=exp.get_parameter('epochs'),
          batch_size=exp.get_parameter('batch-size'),
          validation_data=(x_val, y_val),
          verbose=1)

Epoch 1/10
30/30 [==============================] - 3s 77ms/step - loss: 0.8197 - accuracy: 0.5595 - val_loss: 0.6459 - val_accuracy: 0.6299
Epoch 2/10
30/30 [==============================] - 2s 71ms/step - loss: 0.6831 - accuracy: 0.6113 - val_loss: 0.6014 - val_accuracy: 0.6878
Epoch 3/10
30/30 [==============================] - 2s 72ms/step - loss: 0.6098 - accuracy: 0.6684 - val_loss: 0.5662 - val_accuracy: 0.7164
Epoch 4/10
30/30 [==============================] - 2s 73ms/step - loss: 0.5742 - accuracy: 0.7010 - val_loss: 0.5438 - val_accuracy: 0.7395
Epoch 5/10
30/30 [==============================] - 2s 73ms/step - loss: 0.5442 - accuracy: 0.7264 - val_loss: 0.5195 - val_accuracy: 0.7533
Epoch 6/10
30/30 [==============================] - 2s 72ms/step - loss: 0.5172 - accuracy: 0.7461 - val_loss: 0.4948 - val_accuracy: 0.7699
Epoch 7/10
30/30 [==============================] - 2s 71ms/step - loss: 0.4881 - accuracy: 0.7682 - val_loss: 0.4715 - val_accuracy: 0.7873
Epoch 8/10
30

Logging some things about this experiment:

In [24]:
# log any custom Metric
exp.log_metric('my_custom_metric', 3.14)
    
# log a dataset hash (Hyperparameter)
exp.log_dataset_hash(partial_x_train)

In [25]:
# Logs text: 10 examples from the test dataset with it's labels. These strings appear on the Text Tab in the Comet UI.

for i in range(10):
    exp.log_text(train_examples[i], metadata={"label": train_labels[i].item()})

In [26]:
# Logs the Confusion Matrix

preds = model.predict(test_examples)

# Function to have the labels in vector format    
def onehot(val):
    retval = [0, 0]
    tmp = (val[0] + 1) / 2
    tmp = int(round(tmp))
    tmp = max(min(1, tmp), 0)
    retval[tmp] = 1
    return retval
    
new_preds = [onehot(v) for v in preds]

# Function to show text examples on the Confusion Matrix GUI    
def index_to_example(index):
    text = test_examples[index]
    # data = experiment.log_text(text)
    return {"sample": text.decode(), 
            "assetId": None,
            "type": "string"}
    
exp.log_confusion_matrix(new_preds, 
                         test_labels, 
                         index_to_example_function=index_to_example,
                         file_name="movie-reviews")

782/782 [==============================] - 3s 4ms/step


{'web': 'https://www.comet.com/api/asset/download?assetId=200f3f83c6104884b4916b064cd6a797&experimentKey=3aad1e6f01b246fd8d4ad8f3a2cab846',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=200f3f83c6104884b4916b064cd6a797&experimentKey=3aad1e6f01b246fd8d4ad8f3a2cab846',
 'assetId': '200f3f83c6104884b4916b064cd6a797'}

In [27]:
# Logs Model: The NN in h5 format

registry_model_name ='movie-reviews-nn'

model.save('models/movie-reviews-nn.h5')
exp.log_model(registry_model_name, 'models/movie-reviews-nn.h5')

{'web': 'https://www.comet.com/api/asset/download?assetId=b2cb0a560b284babb0825efe5df04c1d&experimentKey=3aad1e6f01b246fd8d4ad8f3a2cab846',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=b2cb0a560b284babb0825efe5df04c1d&experimentKey=3aad1e6f01b246fd8d4ad8f3a2cab846',
 'assetId': 'b2cb0a560b284babb0825efe5df04c1d'}

In [28]:
exp.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/rodrigomiranda21/rmiranda-movie-reviews/3aad1e6f01b246fd8d4ad8f3a2cab846
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.5595333576202393, 0.819266676902771)
COMET INFO:     batch_accuracy [30]          : (0.521484375, 0.8184523582458496)
COMET INFO:     batch_loss [30]              : (0.40733569860458374, 0.9908294677734375)
COMET INFO:     epoch_duration [10]          : (2.018152774000555, 3.1351911730007487)
COMET INFO:     loss [10]                    : (0.40602511167526245, 0.8196919560432434)
COMET INFO:     my_custom_metric             : 3.14
COMET INFO:     val_accuracy [10]            : (0.6298999786376953, 0.8234999775886536)
COMET INFO:     val_loss [10]                : (0.4078299403190613, 

Register the model on Model Registry

In [ ]:
api = API(api_key="EpnUTFvwqbgvmSpjWOTMBPlP5")

WORKSPACE = 'rodrigomiranda21'
PROJECT_NAME = 'rmiranda-movie-reviews'
EXP_KEY = exp.get_key()

api_exp = api.get_experiment(WORKSPACE, PROJECT_NAME, EXP_KEY)

api_exp.register_model(registry_model_name, version = "1.4.0")

In [29]:
exp.get_key()

'3aad1e6f01b246fd8d4ad8f3a2cab846'